# Pràctica 4 PLH - Rubén Álvarez Aragonés i Pol Pérez Prades

____

# Stuff

## Imports 

In [ ]:
%pip install -r requirements.txt

In [ ]:
import tensorflow as tf
import numpy as np
import nltk
import re
import spacy
from spacy.lang.ca.examples import sentences 
from gensim.models import word2vec
import torch
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')

spacy.cli.download("ca_core_news_sm")


## Entrenament model Word2Vec

In [ ]:
class Word2VecEmbedder:
    def __init__(self, corpus_path, corpus_size, load_model=False, model_path=None):
        if not load_model:
            self.corpus_path = corpus_path
            self.corpus_size = int(corpus_size * 2**30) if corpus_size else None  # Convert GB to bytes
            self.corpus = self.get_corpus(corpus_path)
            self.fit()
        else:
            try:
                self.load(model_path)
            except FileNotFoundError:
                print("Model not found. Please check the path.")
                return

    def get_corpus(self, corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            print("Reading corpus...")
            corpus = f.read(self.corpus_size) if self.corpus_size else f.read()
            print("Preprocessing corpus...")
            corpus = self.preprocess(corpus)  # Preprocess the corpus and tokenize it
        return corpus

    def fit(self, window_size=15, vector_size=300, min_count=10, workers=8, epochs=10):
        # Initialize the Word2Vec model with gensim
        print("Initializing Word2Vec model...")
        self.model = word2vec.Word2Vec(sentences=[self.corpus], vector_size=vector_size, window=window_size, min_count=min_count, workers=workers, epochs=epochs)
        print("Model training completed.")

    def save(self, model_path):
        # Save the model
        self.model.save(model_path)

    def load(self, model_path):
        # Load the model
        self.model = word2vec.Word2Vec.load(model_path)

    def preprocess(self, corpus):
        # Lowercase the corpus
        print("Lowercasing...")
        corpus = corpus.lower()
        
        # Remove special characters
        print("Removing special characters...")
        corpus = re.sub(r'[^a-záàéèíìóòúùñüç\s]', ' ', corpus)
        
        # Tokenize the corpus
        print("Tokenizing...")
        corpus = nltk.word_tokenize(corpus)
        
        # Eliminate last token (probably incomplete word)
        corpus = corpus[:-1]
        
        return corpus

    def get_embedding(self, word):
        # Get the embedding of a word
        try:
            return self.model.wv[word]
        except KeyError:
            print(f"Word '{word}' not in vocabulary.")
            return None

    def print_vocab(self):
        print("Vocabulary:", list(self.model.wv.index_to_key))
        
    

### 1. Model amb 100MB de dades

In [ ]:
Word2Vec_model = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.1)

In [ ]:
Word2Vec_model.save('models/word2vec_model.bin')

In [ ]:
Word2Vec_model.print_vocab()

In [ ]:
Word2Vec_model.get_embedding("de")

In [ ]:
Word2Vec_model.model.wv.most_similar("negre")

### 2. Model amb 500MB de dades

In [ ]:
Word2Vec_model_500 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.5)

In [ ]:
Word2Vec_model.get_embedding("hola")

In [ ]:
Word2Vec_model_500.model.wv.most_similar("inshalla")

### 3. Model amb 1GB de dades

In [ ]:
Word2Vec_model_1024 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 1)

In [ ]:
Word2Vec_model.get_embedding("hola")

### 4. Model amb totes les dades

In [ ]:
Word2Vec_model_full_data = Word2VecEmbedder('corpus\catalan_general_crawling.txt', None)

## Entrenament model de Similitud de Text Semàntic

### Imports

In [ ]:
from importació_data import read_all_ts_data, reformat_data, create_corpus, preprocess, flattened_corpus_count
from importació_data import pair_list_to_x_y
from model_bàsic import build_and_compile_model_better
import tensorflow as tf
from model_bàsic import compute_pearson

- Definim la funció de preprocessament de text tenint en compte les stopwords del català.
- Llegim totes les dades de text similarity dividint-les en train, test i val. 
- Reformatejem les dades per a que siguin l'estructura List[Tuple[str, str, float]]. 
- Definim el corpus i el diccionari amb totes les paraules.
- Creem un diccionari de python amb tots els indexs com a claus i amb la repetició de les paraules com a valor.

In [ ]:
from importació_data import stopwords_cat
stpw_cat = stopwords_cat()

In [ ]:
prepro = lambda x: preprocess(x, stpw_cat)

In [ ]:
train, test, val = read_all_ts_data()
train, test, val = reformat_data(train, test, val)
corpus, all_words = create_corpus(train, test, val, preprocess=prepro)
flat_corpus = flattened_corpus_count(corpus)

### Compartació amb diferents models de Word Embeddings

#### 1. One Hot

In [ ]:
from onehot import map_one_hot

Un embedding OneHot té tamany igual a la llargada del diccionari. En la importació de dades ja hem eliminat les stopwords per reduïr la dimensió, però ara també eliminarem del embedding aquelles paraules que es repeteixen masses poques vegades o massa sovint. Per aconseguir això creem una llista que conté els indexs de les paraules que sí que utilitzarem i la passem com a argument a la funció *map_one_hot()*, per reduïr la dimensió del embedding.

In [ ]:
# Eliminate from all_words, the words that are repeated very few times or too much times
keys_preprocess = [index  for index in all_words if flat_corpus[index] > 10 and flat_corpus[index] < 50]

Convertim les paraules a vectors OneHot amb la funció *map_one_hot()*. Aquesta funció crea un vector de zeros de la mida del diccionari i posa un 1 a la posició de la paraula en el diccionari.

In [ ]:
mapped_one_hot_train = map_one_hot(train, all_words, keys_preprocess)
mapped_one_hot_test = map_one_hot(test, all_words, keys_preprocess)
mapped_one_hot_val = map_one_hot(val, all_words, keys_preprocess)

Separem el X i Y

In [ ]:
x_train_oh, y_train_oh = pair_list_to_x_y(mapped_one_hot_train)
x_val_oh, y_val_oh = pair_list_to_x_y(mapped_one_hot_val)
x_test_oh, y_test_oh = pair_list_to_x_y(mapped_one_hot_test)

Entrenem el model

In [ ]:
batch_size_oh: int = 64
num_epochs_oh: int = 64

train_dataset_oh = tf.data.Dataset.from_tensor_slices((x_train_oh, y_train_oh))
train_dataset_oh = train_dataset_oh.shuffle(buffer_size=len(x_train_oh)).batch(batch_size_oh)

val_dataset_oh = tf.data.Dataset.from_tensor_slices((x_val_oh, y_val_oh))
val_dataset_oh = val_dataset_oh.batch(batch_size_oh)

In [ ]:
embedding_size_oh = len(keys_preprocess)
model_oh = build_and_compile_model_better(embedding_size = embedding_size_oh)
tf.keras.utils.plot_model(model_oh, show_shapes=True, show_layer_activations=True, )
print(model_oh.summary())

In [ ]:
model_oh.fit(train_dataset_oh, epochs=num_epochs_oh, validation_data=val_dataset_oh)

Evaluem el model amb la partició de validació

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_oh, x_train_oh, y_train_oh)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_oh, x_val_oh, y_val_oh)}")

Provem el model amb la partició de test

In [ ]:
print(f"Correlación de Pearson (test): {compute_pearson(model_oh, x_test_oh, y_test_oh)}")

#### 2. Word2Vec preentrenats 

In [ ]:
from word2vec_tf_idf import map_pairs_w2v

In [ ]:
WORD_EMBEDDING_FILE = ""


USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.es.gensim.bin'
    # wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')
else:
    from gensim.models import fasttext
    wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

##### 2.1 Mitjana

In [ ]:
mapped_w2v_mean_train = map_pairs_w2v(train, wv_model, dictionary=all_words, preprocess=prepro)
mapped_w2v_mean_test = map_pairs_w2v(test, wv_model, dictionary=all_words, preprocess=prepro)
mapped_w2v_mean_val = map_pairs_w2v(val, wv_model, dictionary=all_words, preprocess=prepro)

In [ ]:
x_train_w2v_mean, y_train_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_train)
x_val_w2v_mean, y_val_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_val)
x_test_w2v_mean, y_test_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_test)

Entrenem el model

In [ ]:
batch_size_w2v_mean: int = 64
num_epochs_w2v_mean: int = 64

train_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_train_w2v_mean, y_train_w2v_mean))
train_dataset_w2v_mean = train_dataset_w2v_mean.shuffle(buffer_size=len(x_train_w2v_mean)).batch(batch_size_w2v_mean)

val_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_val_w2v_mean, y_val_w2v_mean))
val_dataset_w2v_mean = val_dataset_w2v_mean.batch(batch_size_w2v_mean)

In [ ]:
embedding_size_w2v_mean = 300
model_w2v_mean = build_and_compile_model_better(embedding_size = embedding_size_w2v_mean)
tf.keras.utils.plot_model(model_w2v_mean, show_shapes=True, show_layer_activations=True)
print(model_w2v_mean.summary())

In [ ]:
model_w2v_mean.fit(train_dataset_w2v_mean, epochs=num_epochs_w2v_mean, validation_data=val_dataset_w2v_mean)

Evaluem el model amb la partició de validació

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_mean, x_train_w2v_mean, y_train_w2v_mean)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_mean, x_val_w2v_mean, y_val_w2v_mean)}")

Provem el model amb la partició de test

In [ ]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_mean, x_test_w2v_mean, y_test_w2v_mean)}")

##### 2.2 Tf-idf

In [ ]:
from gensim.models import TfidfModel
modelo_tfidf = TfidfModel(corpus)

In [ ]:
mapped_w2v_tfidf_train = map_pairs_w2v(train, wv_model, dictionary=all_words,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_test = map_pairs_w2v(test, wv_model, dictionary=all_words,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_val = map_pairs_w2v(val, wv_model, dictionary=all_words,tf_idf_model=modelo_tfidf, preprocess=prepro)

In [ ]:
x_train_w2v_tfidf, y_train_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_train)
x_val_w2v_tfidf, y_val_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_val)
x_test_w2v_tfidf, y_test_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_test)

Entrenem el model

In [ ]:
batch_size_w2v_tfidf: int = 64
num_epochs_w2v_tfidf: int = 64

train_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_train_w2v_tfidf, y_train_w2v_tfidf))
train_dataset_w2v_tfidf = train_dataset_w2v_tfidf.shuffle(buffer_size=len(x_train_w2v_tfidf)).batch(batch_size_w2v_tfidf)

val_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_val_w2v_tfidf, y_val_w2v_tfidf))
val_dataset_w2v_tfidf = val_dataset_w2v_tfidf.batch(batch_size_w2v_tfidf)

In [ ]:
embedding_size_w2v_tfidf = 300
model_w2v_tfidf = build_and_compile_model_better(embedding_size = embedding_size_w2v_tfidf)
tf.keras.utils.plot_model(model_w2v_tfidf, show_shapes=True, show_layer_activations=True)
print(model_w2v_tfidf.summary())

In [ ]:
model_w2v_tfidf.fit(train_dataset_w2v_tfidf, epochs=num_epochs_w2v_tfidf, validation_data=val_dataset_w2v_tfidf)

Evaluem el model amb la partició de validació

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_tfidf, x_train_w2v_tfidf, y_train_w2v_tfidf)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_tfidf, x_val_w2v_tfidf, y_val_w2v_tfidf)}")

Provem el model amb la partició de test

In [ ]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_tfidf, x_test_w2v_tfidf, y_test_w2v_tfidf)}")

#### 3. SpaCy

In [ ]:
import spacy
from spacy_embed import map_spacy_embed

In [ ]:
!python -m spacy download ca_core_news_md

In [ ]:
spacy_model = spacy.load("ca_core_news_md")

Passem les particions de train, test i val a vectors amb la funció *map_spacy()*. Aquesta funció utilitza el model de SpaCy per a convertir les paraules a vectors.

In [ ]:
mapped_spacy_train = map_spacy_embed(train, spacy_model)
mapped_spacy_test = map_spacy_embed(test, spacy_model)
mapped_spacy_val = map_spacy_embed(val, spacy_model)

Dividim les particions en X i Y

In [ ]:
x_train_sp, y_train_sp = pair_list_to_x_y(mapped_spacy_train)
x_val_sp, y_val_sp = pair_list_to_x_y(mapped_spacy_val)
x_test_sp, y_test_sp = pair_list_to_x_y(mapped_spacy_test)

Entrenem el model

In [ ]:
batch_size_sp: int = 64
num_epochs_sp: int = 64

train_dataset_sp = tf.data.Dataset.from_tensor_slices((x_train_sp, y_train_sp))
train_dataset_sp = train_dataset_sp.shuffle(buffer_size=len(x_train_sp)).batch(batch_size_sp)

val_dataset_sp = tf.data.Dataset.from_tensor_slices((x_val_sp, y_val_sp))
val_dataset_sp = val_dataset_sp.batch(batch_size_sp)

In [ ]:
embedding_size_sp = 300
model_sp = build_and_compile_model_better(embedding_size = embedding_size_sp)
tf.keras.utils.plot_model(model_sp, show_shapes=True, show_layer_activations=True, )
print(model_sp.summary())

In [ ]:
model_sp.fit(train_dataset_sp, epochs=num_epochs_sp, validation_data=val_dataset_sp)

Evaluem el model amb al partició de validation

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_train_sp, y_train_sp)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_sp, x_val_sp, y_val_sp)}")

Provem el model amb la partició de test

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_test_sp, y_test_sp)}")

#### 4. RoBERTa 

In [ ]:
nlp = spacy.load("ca_core_news_trf")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

#### 5. RoBERTa fine-tuned

In [ ]:
model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)